In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import mlflow
import mlflow.sklearn

# Set the MLflow tracking URI to the Docker container's exposed port
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Set the experiment name
mlflow.set_experiment("Iris_KNN_Experiment")

# Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Start an MLflow run
with mlflow.start_run() as run:
    # Initialize the model
    knn = KNeighborsClassifier(n_neighbors=3)

    # Train the model
    knn.fit(X_train, y_train)

    # Make predictions
    y_pred = knn.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy:.2f}')

    # Print classification report
    print('Classification Report:')
    print(classification_report(y_test, y_pred, target_names=iris.target_names))

    # Print confusion matrix
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))

    # Log parameters and metrics to MLflow
    mlflow.log_param("n_neighbors", 3)
    mlflow.log_metric("accuracy", accuracy)

    # Log the model to MLflow
    joblib_file = "model.pkl"
    joblib.dump(knn, joblib_file)
    mlflow.log_artifact(joblib_file)
    mlflow.sklearn.log_model(knn, "model")

    # Register the model to the MLflow model registry
    model_name = "KNN_Iris"
    mlflow.register_model(f"runs:/{run.info.run_id}/model", model_name)

    print(f'Model saved to {joblib_file} and registered as {model_name}')


2024/06/26 15:03:57 INFO mlflow.tracking.fluent: Experiment with name 'Iris_KNN_Experiment' does not exist. Creating a new experiment.


Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       1.00      1.00      1.00        13
   virginica       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

Confusion Matrix:
[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]


Successfully registered model 'KNN_Iris'.
2024/06/26 15:04:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: KNN_Iris, version 1


Model saved to model.pkl and registered as KNN_Iris


Created version '1' of model 'KNN_Iris'.
